**Models Prophet**

In [108]:
#%pip install prophet

In [109]:
import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import OneHotEncoder
import pickle
import os
import numpy as np
from datetime import timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



In [110]:
# Carreguem les dades
dades_temporals = pd.read_csv('daily_covidMatrix.csv') 
dades_estatiques = pd.read_csv('static_stateMatrix.csv')
clusters_targets = pd.read_csv('state_clusters_0.7-0.3.csv') 

In [111]:
targets = {
    'positiveIncrease': 'Cluster_positiveIncrease',
    'hospitalizedIncrease': 'Cluster_hospitalizedIncrease',
    'deathIncrease': 'Cluster_deathIncrease'
}

In [112]:
dades = pd.merge(dades_temporals, dades_estatiques, on='state', how='left')

In [113]:
# Inicialitzem encoder per l'estat
encoder = OneHotEncoder(sparse=False)
states_encoded = encoder.fit_transform(dades[['state']])
state_columns = encoder.get_feature_names_out(['state'])
states_encoded_df = pd.DataFrame(states_encoded, columns=state_columns)

dades = pd.concat([dades.reset_index(drop=True), states_encoded_df.reset_index(drop=True)], axis=1)


C:\Users\mibks\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [114]:
altres_regressors = [
    'no_coverage', 'bedsState_local_government','bedsNon_profit','bedsFor_profit',
    'bedsTotal','population_state','pop_density_state', 'Low_SVI_CTGY',
    'Moderate_Low_SVI_CTGY','Moderate_High_SVI_CTGY', 'Metro',
    'private_coverage','public_coverage','labor_cov_diff','pop_60-69',
    'pop_70-79','pop_80+'
]
#,'High_SVI_CTGY','Non-metro', 'pop_0-9', 'pop_10-19','pop_20-29','pop_30-39','pop_40-49','pop_50-59',



In [115]:
# Dividir en train i test

dades['date'] = pd.to_datetime(dades['date'])

train = dades[dades['date'] < '2021-03-01']
test = dades[dades['date'] >= '2021-03-01']

Entrenar els diferents models

In [116]:
# Crear la carpeta per guardar els models
os.makedirs('models_prophet', exist_ok=True)

# Iterar per cada variable target
for target_var, cluster_col in targets.items():
    # Tots els clusters possibles per aquesta variable
    clusters = clusters_targets[cluster_col].unique()
    
    for cluster in clusters:
        # Estats que pertanyen a aquest cluster
        estats_cluster = clusters_targets[clusters_targets[cluster_col] == cluster]['State'].unique()
        
        dades_cluster = train[train['state'].isin(estats_cluster)]      
          
        columns_model = ['date', target_var] + list(state_columns) + altres_regressors
        df_prophet = dades_cluster[columns_model].rename(columns={'date': 'ds', target_var: 'y'})
        
        model = Prophet(
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.0)
        
        regressors = list(state_columns) + altres_regressors              
        for reg in regressors:
            model.add_regressor(reg)

        model.fit(df_prophet)
            
        model_filename = f'models_prophet/prophet_{target_var}_cluster{cluster}.pkl'
        with open(model_filename, 'wb') as f:
            pickle.dump(model, f)
        
        print(f"Model entrenat i guardat per {target_var}, cluster {cluster}")


23:21:38 - cmdstanpy - INFO - Chain [1] start processing
23:21:38 - cmdstanpy - INFO - Chain [1] done processing


Model entrenat i guardat per positiveIncrease, cluster 17


23:21:38 - cmdstanpy - INFO - Chain [1] start processing
23:21:39 - cmdstanpy - INFO - Chain [1] done processing


Model entrenat i guardat per positiveIncrease, cluster 21


23:21:39 - cmdstanpy - INFO - Chain [1] start processing
23:21:39 - cmdstanpy - INFO - Chain [1] done processing
23:21:40 - cmdstanpy - INFO - Chain [1] start processing


...

Model entrenat i guardat per deathIncrease, cluster 11


23:27:01 - cmdstanpy - INFO - Chain [1] start processing
23:27:01 - cmdstanpy - INFO - Chain [1] done processing
23:27:01 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
23:27:02 - cmdstanpy - INFO - Chain [1] start processing
23:27:10 - cmdstanpy - INFO - Chain [1] done processing


Model entrenat i guardat per deathIncrease, cluster 14


23:27:10 - cmdstanpy - INFO - Chain [1] start processing
23:27:10 - cmdstanpy - INFO - Chain [1] done processing
23:27:10 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
23:27:10 - cmdstanpy - INFO - Chain [1] start processing
23:28:52 - cmdstanpy - INFO - Chain [1] done processing


Model entrenat i guardat per deathIncrease, cluster 20


Predir el test

In [117]:
metriques_totals = []

# Iterar per cada target
for target_var, cluster_col in targets.items():
    # Tots els clusters possibles per aquest target
    clusters = clusters_targets[cluster_col].unique()
    
    for cluster in clusters:
        model_filename = f'models_prophet/prophet_{target_var}_cluster{cluster}.pkl'
        
        with open(model_filename, 'rb') as f:
            model = pickle.load(f)
        
        estats_cluster = clusters_targets[clusters_targets[cluster_col] == cluster]['State'].unique()
        
        for estat in estats_cluster:
            dades_estat = train[train['state'] == estat]
            if dades_estat.empty:
                continue
            
            columns_model = ['date', target_var] + list(state_columns) + altres_regressors
            df_prophet = dades_estat[columns_model].rename(columns={'date': 'ds', target_var: 'y'})
            df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
            
            # Última fila coneguda de regressors
            regressors = list(state_columns) + altres_regressors
            last_known = df_prophet[regressors].iloc[-1]
            #print(estat)
            

            # Genera les dates futures (7 dies a partir de l'última data coneguda)
            start_date = df_prophet['ds'].max() + timedelta(days=1)
            future_dates = pd.date_range(start=start_date, periods=7, freq='D')
            future = pd.DataFrame({'ds': future_dates})

            # Repetir últims valors de regressors
            future_states_df = pd.DataFrame([last_known.values] * 7, columns=regressors).reset_index(drop=True)
            future = pd.concat([future, future_states_df], axis=1)
            #print('------------------------------')
            #print(future)
            
            # Predicció
            forecast = model.predict(future)
            #print('------------------')
            #print(forecast)
            prediccions = forecast[['ds', 'yhat']].copy()
            prediccions['state'] = estat
            prediccions['target'] = target_var
            #print('------------------')
            #print(prediccions)

            # Comparació 
            test_estat = test[
                (test['state'] == estat) &
                (test['date'].isin(prediccions['ds']))
            ]       
                      
            comparacio = pd.merge(prediccions, test_estat, left_on=['ds', 'state'], right_on=['date', 'state'])
            
            y_true = comparacio[target_var]
            y_pred = comparacio['yhat']
            
            # Mètriques
            mae = mean_absolute_error(y_true, y_pred)
            mse = mean_squared_error(y_true, y_pred)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_true, y_pred)
           
            metriques_totals.append({
                'target': target_var,
                'state': estat,
                'cluster': cluster,
                'MAE': mae,
                'MSE': mse,
                'R2': r2
            })
                
df_metriques = pd.DataFrame(metriques_totals)



In [118]:
df_metriques['model'] = df_metriques['target'] + '_cluster' + df_metriques['cluster'].astype(str)

mitjanes = df_metriques.groupby('model').agg({
    'MAE': 'mean',
    'MSE': 'mean',
    'R2': 'mean'
}).reset_index()
mitjanes = mitjanes.rename(columns=lambda x: f'mitjana_{x}' if x != 'model' else x)

# Millor resultat per model (mínim MAE, mínim MSE, màxim R2)
millors = df_metriques.groupby('model').agg({
    'MAE': 'min',
    'MSE': 'min',
    'R2': 'max'
}).reset_index()
millors = millors.rename(columns=lambda x: f'millor_{x}' if x != 'model' else x)

# Pitjor resultat per model (màxim MAE, màxim MSE, mínim R2)
pitjors = df_metriques.groupby('model').agg({
    'MAE': 'max',
    'MSE': 'max',
    'R2': 'min'
}).reset_index()
pitjors = pitjors.rename(columns=lambda x: f'pitjor_{x}' if x != 'model' else x)

# Juntar-ho tot en un sol DataFrame final
resultats_models = mitjanes.merge(millors, on='model').merge(pitjors, on='model')

resultats_models


,model,mitjana_MAE,mitjana_MSE,mitjana_R2,millor_MAE,millor_MSE,millor_R2,pitjor_MAE,pitjor_MSE,pitjor_R2
0,deathIncrease_cluster1,33.641582,1.331696e+03,-4.389769,24.711521,7.497009e+02,-0.812834,46.142955,2.218391e+03,-9.643581
1,deathIncrease_cluster10,12.695371,2.107473e+02,-3.610056,11.369993,1.715329e+02,0.000106,14.020749,2.499617e+02,-7.220217
2,deathIncrease_cluster11,56.077193,5.947148e+03,0.342833,56.077193,5.947148e+03,0.342833,56.077193,5.947148e+03,0.342833
3,deathIncrease_cluster12,53.787953,3.451689e+03,-22.916847,31.140102,1.806262e+03,0.028998,65.592248,4.344751e+03,-76.641425
4,deathIncrease_cluster13,151.318958,2.917965e+04,-34.493661,88.138360,1.063126e+04,-1.198767,214.499556,4.772803e+04,-67.788555
5,deathIncrease_cluster14,87.664621,1.779077e+04,-0.381319,87.664621,1.779077e+04,-0.381319,87.664621,1.779077e+04,-0.381319
6,deathIncrease_cluster15,140.125759,2.474822e+04,-69.317871,103.384538,1.083094e+04,-0.193761,176.866979,3.866549e+04,-138.441982
7,deathIncrease_cluster16,1.729979,3.661098e+00,-4.276288,1.729979,3.661098e+00,-4.276288,1.729979,3.661098e+00,-4.276288
8,deathIncrease_cluster17,11.008761,1.518950e+02,-40.259618,8.994545,8.935463e+01,-0.438179,13.022977,2.144354e+02,-80.081058
9,deathIncrease_cluster18,32.969776,2.084202e+03,-9.788283,16.436543,3.297783e+02,0.092427,53.054857,4.996281e+03,-29.035567


In [119]:
#resultats_models.to_csv("resultats2_0.3-0.7.csv", index=False)
#resultats_models.to_csv("resultats2_0.5-0.5.csv", index=False)
resultats_models.to_csv("resultats2_0.7-0.3.csv", index=False)
#resultats_models.to_csv("resultats2.csv", index=False)